# 1. Motivación: Problema de los N-cuerpos

Una región del espacio contiene N partículas que interactuan gravitacionalmente entre sí. El campo gravitacional genera una aceleración en las partículas, lo que cambia su posición, de la cuál depende el campo. Esta relación se expresa como un sistema de ecuaciones diferenciales de segundo orden: 

\begin{align}
    m_i\ddot{\vec{r}_i} = \sum^N_{j=1, j\neq i}G m_im_j\frac{\vec{r}_j-\vec{r}_i}{|\vec{r}_j-\vec{r}_i|^3}
\end{align}

Implementar este cálculo a "fuerza bruta" resulta en que el número de operaciones necesarias es:
\begin{align}
        \frac{1}{2}N(N-1) \approx \mathcal{O}\left(N^2\right)
\end{align}

# 2. Algoritmo Barnes-Hut
Es un método para agrupar partículas cercanas. Cuando el grupo se encuentra lo suficientemente lejos de una partícula, la fuerza ejercida sobre esta última puede aproximarse por aquella producida por el centro de masa del gurpo.

#### $\circ$ Quadtree y Octotree:
| ![title](Images/octotree.png)|
|:--:|
| <b>Partición del espacio bidimencional y representación en el quadtree. Tomado de [1]</b>|


#### $\circ$ Centro de Masa:
\begin{align}
    \vec{r}_{CM} = \frac{\sum^N_{i=1}m_i\vec{r}_i}{\sum^N_{i=1}m_i}\\
\end{align}

En el cálculo de la fuerza sobre una partícula, se encuentra la razón entre el lado de la caja que representa el nodo (<i>d</i>) y la distancia de la partícula al centro de masa del nodo (<i>r</i>): 
\begin{align}
    \theta = \frac{d}{r}
\end{align}
Si $\theta$ es menor que cierto valor de tolerancia, entonces se puede usar el centro de masa del nodo como aproximación a todas las partículas que contiene. El orden del algoritmo resultante es:
\begin{align}
    \mathcal{O}(N\log N)
\end{align}

# 3. Implementación:
## $\circ$ Creación del árbol

In [ ]:
class Node:
    '''---------------------------------------------------------------------
    A node object will represent a body (if node.child is None)
    or an abstract node of the octant-tree if it has node.child attributes.
    ----------------------------------------------------------------------'''
    def __init__(self, m, position, momentum):
        '''-------------------------------------------------------
        Creates a child-less node using the arguments
        ----------------------------------------------------------
        .mass     : scalar
        .position : NumPy array  with the coordinates [x,y,z]
        .momentum : NumPy array  with the components [px,py,pz]
        -------------------------------------------------------'''
        self.m = m
        self.m_pos = m * position #Mass times position. Usefull in calculating center of mass
        self.momentum = momentum
        self.child = None
        self.force = None         #Force on the node    
    
    def position(self):
        '''------------------------------------------
        Returns the physical coordinates of the node.
        -------------------------------------------'''
        return self.m_pos / self.m
    
    def reset_location(self):
        '''-----------------------------------------------------
        Resets the position of the node to the 0th-order octant.
        The size of the octant is reset to the value 1.0
        ------------------------------------------------------'''
        self.size = 1.0
        # The relative position inside the 0th-order octant is equal
        # to the current physical position
        self.relative_position = self.position().copy()
        
    def subdivide(self, i):
        '''-------------------------------------------------------------------
        Places the node node into the next order octant along the direction i
        and recalculates the relative_position of the node inside this octant.
        --------------------------------------------------------------------'''
        self.relative_position[i] *= 2.0
        if self.relative_position[i] < 1.0:
            octant = 0
        else:
            octant = 1
            self.relative_position[i] -= 1.0
        return octant
    def place_into_octant(self):
        '''-------------------------------------------------------------
        Places the node into next order octant.
        Returns the octant number according to the labels defined in the
        documentation.
        --------------------------------------------------------------'''
        # The next order octant will have half the size of the current octant
        self.size = 0.5 * self.size
        return self.subdivide(2) + 2*self.subdivide(1) + 4*self.subdivide(0)

| ![title](Images/tree.png)|
|:--:|
| <b>Construcción del quadtree. Tomado de [1].</b>|

Al agregar un cuerpo a un nodo del árbol existen tres posibilidades:

<ol>
  <li>El nodo esta vacío: Poner el cuerpo allí.</li>
  <li>El nodo es un nodo interno: Actualizar centro de masa y masa total. Añadir el cuerpo al cuadrante apropiado.</li>
  <li>El nodo ya contiene un cuerpo: Volver el nodo un nodo interno e insertar ambos cuerpos en los cuadrantes apropiados. Luego, actualizar centro de masa y masa total.</li>
</ol>

In [ ]:
def add(body, node):
    '''------------------------------------------------------------
    Defines the octo-tree by introducing a body and locating it
    according to three conditions (see documentation for details).
    Returns the updated node containing the body.
    ------------------------------------------------------------'''
    smallest_quadrant = 1.e-4 # Lower limit for the size of the octants
    # Case 1. If node does not contain a body, the body is put in here
    new_node = body if node is None else None
    
    if node is not None and node.size > smallest_quadrant:
        # Case 3. If node is an external node, then the new body can not
        # be put in there. We have to verify if it has .child attribute
        if node.child is None:
            new_node = deepcopy(node)
            # Subdivide the node creating 8 children
            new_node.child = [None for i in range(8)]
            # Place the body in the appropiate octant
            octant = node.place_into_octant()
            new_node.child[octant] = node
        # Case 2. If node is an internal node, it already has .child attribute
        else:
            new_node = node
        # For cases 2 and 3, it is needed to update the mass and the position
        # of the node
        new_node.m += body.m
        new_node.m_pos += body.m_pos
        # Add the new body into the appropriate octant.
        octant = body.place_into_octant()
        new_node.child[octant] = add(body, new_node.child[octant])
    return new_node

## $\circ$ Cálculo de la fuerza

\begin{align}
    F_{j\to i} = G m_im_j\frac{\vec{r}_j-\vec{r}_i}{|\vec{r}_j-\vec{r}_i|^3}
\end{align}

In [1]:
def distance_between(node1, node2):
    '''--------------------------------------------------------
    Returns the distance between node1 and node2. (scaled down)
    ---------------------------------------------------------'''
    return norm(node1.position() - node2.position())

def gravitational_force(node1, node2):
    '''--------------------------------------------------------------
    Returns the gravitational force that node1 exerts on node2.
    A short distance cutoff is introduced in order to avoid numerical
    divergences in the gravitational force.
    ---------------------------------------------------------------'''
    cutoff_dist = 2.e-4
    d = distance_between(node1, node2)
    if d < cutoff_dist:
        return array([0., 0., 0.])
    else:
        return G*node1.m*node2.m*(node1.position() - node2.position())/d**3/scale_factor**2

| ![title](Images/Force.png)|
|:--:|
| <b>Cálculo de la fuerza actuando sobre el cuerpo $\textit{a}$. Tomado de [1].</b>|


In [ ]:
def force_on(body, node, theta):
    '''-------------------------------------------------------------------------
    # Barnes-Hut algorithm: usage of the octo-tree. This function computes
    # the net force on a body exerted by all bodies in node "node".
    # Note how the code is shorter and more expressive than the human-language
    # description of the algorithm.
    -------------------------------------------------------------------------'''
    # 1. If the current node is an external node,
    #    calculate the force exerted by the current node on b.
    if node.child is None:
        return gravitational_force(node,body)#node.force_on(body)

    # 2. Otherwise, calculate the ratio s/d. If s/d < θ, treat this internal
    #    node as a single body, and calculate the force it exerts on body b.
    if node.size < distance_between(node,body)*theta:#node.distance(body) * theta:
        return gravitational_force(node,body)#node.force_on(body)
        
    # 3. Otherwise, run the procedure recursively on each child.
    return sum(force_on(body, c, theta) for c in node.child if c is not None)

## $\circ$ Evolución en el tiempo

#### $\circ$ Sistema EDO:
\begin{align*}
    m\frac{d\vec{r}}{dt} &= m\vec{v}\\
    m\frac{d\vec{v}}{dt} &= \sum^N_{j=1, j\neq i}G m_im_j\frac{\vec{r}_j-\vec{r}_i}{|\vec{r}_j-\vec{r}_i|^3}
\end{align*}
#### $\circ$ Método de Euler:
\begin{align*}
r(t_{i+1}) &= r(t_{i}) + h f(t_i,r(t_i)) +  \mathcal{O}(h^2)
\end{align*}

In [ ]:
def step(bodies, root, theta, dt):
    '''------------------------------
    Euler method for time evolution.
    -------------------------------'''
    for body in bodies:
        body.force = force_on(body, root, theta)
        body.momentum += body.force*dt
        body.m_pos += scale_factor*body.momentum*dt

## $\circ$ Flujo del código

In [ ]:
def system_init(N, max_mass, BHM, center, BHmomentum, ini_radius, alpha, beta):
    '''-------------------------------------------------------------------
    Initializes the N-body system by defining the position and momentum
    of the bodies and creating the corresponding objects of the Node class
    --------------------------------------------------------------------'''
    #Defines initial conditions
    bodies = []
    bodies.append(Node(BHM, position=center, momentum=BHmomentum))   
    masses, positions, momenta = spiral_galaxy(N, max_mass, BHM, center, ini_radius, alpha, beta)
    #Creates nodes
    for i in range(N-1):
        bodies.append(Node(masses[i], positions[i], momenta[i]))
    return bodies

In [ ]:
def evolve(bodies, n, center, ini_radius, img_step, image_folder='images/', video_name='my_video.mp4'):
    '''---------------------------------------------------------------------------------------------
    This function evolves the system in time using the Euler algorithm and the Barnes-Hut octo-tree
    ----------------------------------------------------------------------------------------------'''
    # Principal loop over time iterations.
    for i in range(n+1):
        # The octo-tree is recomputed at each iteration.
        root = None
        for body in bodies:
            body.reset_location()
            root = add(body, root) 
        # Evolution using the integration method
        step(bodies, root, theta, dt)
        # Write the image files
        if i%img_step==0:
            print("Writing image at time {0}".format(i))
            plot_bodies(bodies, i//img_step,image_folder)

## $\circ$ Condiciones iniciales 
### $\circ$ Velocidad Kepleriana:
\begin{align*}
    |\vec{v}| = \sqrt{\frac{GM_B}{r}}
\end{align*}
### $\circ$ Plano Galáctico:
| ![title](Images/Plane.png)|
|:--:|
| <b>Orientación del plano del dico galáctico. Tomado de https://ui.adsabs.harvard.edu/abs/2017A%26A...599L...7L/abstract.</b>|


### $\circ$ Distribución de Estrellas:
Se usa la distribución superficial de luminosidad de la galaxia como proporcional a la densidad superficial de estrellas. Se distinguen dos regiones:
<ol>
    <li> $\textbf{Bulbo}$: Grupo compacto de estrellas en el centro galáctico similar a galaxias elípticas. Siguen la ley de Vaucoulerus:
    \begin{align*}\\
        I(r) = I_0\exp\left(-kr^{1/4}\right)\\ \\
    \end{align*}</li>
    <li> $\textbf{Disco}$: La distribución superficial de luminosidad puede describirse con una ley exponencial:
    \begin{align*}\\
        I(r) = I_0\exp\left(-\frac{r}{r_D}\right)\\
    \end{align*}</li>
</ol>

| ![title](Images/Radius.png)|
|:--:|
| <b>Distribución de brillo superficial en una galaxia espiral según Freeman (Disco) y Vaucouleurs (Bulbo). Tomado de [3].</b>|

Se usa el método de transformada inversa para mapear puntos $p_0$ de una distribución uniforme aleatoria a los de la distribución deseada $f(t)$ resolviendo para $x$ la siguiente expresión:
\begin{align*}
    \int^x_0f(t)dt - p_0 = 0
\end{align*}

In [ ]:
def func(x,Distribution,Point): 
    """------------------------------------------------------------------------
    Equation that follows the point of the wanted distribution that matches the 
    random one of a uniform distribution
    ---------------------------------------------------------------------------
       x            : Random variable in the wanted distribution (unkonwn)
       Distribution : Wanted distribution
       Point        : Random variable in the uniform distribution
    ------------------------------------------------------------------------"""
    return integrate.quad(Distribution,0,x)[0]-Point

def spiral_galaxy(N, max_mass, BHM, center, ini_radius, alpha, beta):
    '''-----------------------------------------------------------------------
    Use a radial distrubution of masses proportional to the brightness surface
    distributation to create a plain Bulb and Disk resembling an spiral galaxy
    --------------------------------------------------------------------------
       N            : Number of particles
       max_mass     : Biggest mass of the stars in the system 
       BHM          : Black Hole's mass
       center       : Black Hole position
       ini_radius   : Galaxy radius
       alpha        : Angle in the x,y plane
       beta         : Inclination
    ------------------------------------------------------------------------'''
    N -= 1
    random.seed(10)
    # Generates N random particles 
    positions = empty([N,3])
    momenta = empty([N,3])
    # Random masses varies between 1 solar mass and max_mass solar masses
    masses = random.random(N)*(max_mass-1.) + 1.
    #Parameters of the model of density of starts
    initial_density=.1
    const_bulb=.3
    const_disc=.8
    bulb_radius=0.2
    #Model of density normalized
    f1 = lambda x: initial_density*exp(-x**(1/4)/const_bulb)        #Bulge
    f2 = lambda x: f1(bulb_radius)*exp(-(x-bulb_radius)/const_disc) #Disc
    f = lambda x:  f1(x) if x<bulb_radius else f2(x)                #Piecewise 
    norm = integrate.quad(f,0,1)[0]                                  
    uf=lambda x: f(x)/norm                                          #Density function with integral=1
    #Random angle generation
    gamma = random.random(N)*2*pi
    #Random width
    width = .05*ini_radius                                          #Half of with in relation to the radius of the galaxy
    gross  = random.random(N)*2*width-width
    temp = beta
    #Uniform distribution to get random points
    Uniform = random.random(N)
    #Empty array for the points mapped from the uniform distribution
    Map=zeros(N)   
    for i in range(N):
        #Calls the function that maps the ramdom points to the wanted distribution for the radius 
        Map[i]=fsolve(func,0,args=(uf,Uniform[i]))*ini_radius
        #Creates an elipsoid in the region of the bulge
        if Map[i] < bulb_radius*ini_radius:
            a = 0.18*ini_radius
            bulg_countour = a*sqrt(1-(Map[i]/(bulb_radius*ini_radius))**2)
            gross[i] = random.random(1)*2*bulg_countour-bulg_countour
        #Adjustment for width
        beta += arctan(gross[i]/Map[i])
        Map[i] = sqrt(Map[i]**2+gross[i]**2)
        #Change to cartesian coordinates
        positions[i][0] = scale_factor*Map[i]*(cos(gamma[i])*cos(alpha)+
                                   sin(gamma[i])*cos(beta)*sin(alpha)) + center[0]
        positions[i][1] = scale_factor*Map[i]*(sin(gamma[i])*cos(beta)*cos(alpha)-
                                   cos(gamma[i])*sin(alpha))+ center[1]
        positions[i][2] = scale_factor*Map[i]*sin(gamma[i])*sin(beta) + center[2]
        # Keplerina velocity in the plain of the disc 
        Kep_v = sqrt(G*BHM/Map[i])
        vec_vel=array([-Map[i]*(sin(gamma[i])*cos(alpha)-cos(gamma[i])*cos(beta)*sin(alpha)),
                       Map[i]*(cos(gamma[i])*cos(beta)*cos(alpha)+sin(gamma[i])*sin(alpha)), 
                       Map[i]*cos(gamma[i])*sin(beta)])/Map[i]
        momenta[i][0] = masses[i]*Kep_v*vec_vel[0]
        momenta[i][1] = masses[i]*Kep_v*vec_vel[1]
        momenta[i][2] = masses[i]*Kep_v*vec_vel[2]
        beta = temp
    return masses, positions, momenta

# A futuro cercano y medio...

<oi>
    <li> La dinámica gravitacional sin colisiones es más adecuada para describir la materia oscura que la bariónica. Se desea usar este código para modelor el halo de materia oscura que rodea una galaxia espiral.</li>
    <li> Se debe implementar "softening" de la fuerza gravitacional para partículas cercanas.</li>
    <li> Se considera usar un paso de tiempo adaptativo dependiento de la densidad local de partículas. </li>
    <li> Usar un integrador simpléctico. </li>
    <li> Refinar el uso del árbol mediante expansión multipolar (Fast Multipole Method, FMM).</li>
    <li> Usar un esquema híbrido entre un método de malla (distancias lejanas)  y  FMM (distancias cercanas).</li>
    <li> Se considera si implementar el código (o partes de este) en C++.</li>
    <li> Implementar paralelización.</li>
    <li> Pulir la forma en que se guardan los datos y se produce la animación </li>
</oi>

## References

1. [Barnes-Hut Galaxy Simulator assignment](https://www.cs.princeton.edu/courses/archive/fall03/cs126/assignments/barnes-hut.html)
2. [The Barnes-Hut Galaxy Simulator](https://beltoforion.de/en/barnes-hut-galaxy-simulator/)
3. [Rendering a Galaxy with the density wave theory](https://beltoforion.de/en/spiral_galaxy_renderer/index.php/)
4. Vogelsberger, M., Marinacci, F., Torrey, P., & Puchwein, E. (2020). Cosmological simulations of galaxy formation. Nature Reviews Physics, 2(1), 42–66. https://doi.org/10.1038/s42254-019-0127-2
5. Graps, A. (1996). N-Body/Particle Simulation Methods. . https://www.cs.cmu.edu/afs/cs/academic/class/15850c-s96/www/nbody.html